In [1]:
import numpy as np
import torch
import torch.nn as nn
from utility_funcs import get_train_labels_test
from scipy import stats
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd

from sklearn.preprocessing import StandardScaler


In [2]:
!nvidia-smi

Thu Apr 14 22:42:20 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 512.15       Driver Version: 512.15       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   46C    P8     5W /  N/A |      0MiB /  6144MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
data_path = "../Data/"
train = pd.read_csv(data_path + "train.csv")
labels = pd.read_csv(data_path + "train_labels.csv")
test = pd.read_csv(data_path + "test.csv")


In [4]:
train.head()


,sequence,subject,step,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,sensor_09,sensor_10,sensor_11,sensor_12
0,0,47,0,-0.196291,0.112395,1.0,0.329204,-1.004660,-0.131638,-0.127505,0.368702,-0.1,-0.963873,-0.985069,0.531893,4.751492
1,0,47,1,-0.447450,0.134454,1.0,-0.658407,0.162495,0.340314,-0.209472,-0.867176,0.2,-0.301301,0.082733,-0.231481,0.454390
2,0,47,2,0.326893,-0.694328,1.0,0.330088,0.473678,1.280479,-0.094718,0.535878,1.4,1.002168,0.449221,-0.586420,-4.736147
3,0,47,3,0.523184,0.751050,1.0,0.976991,-0.563287,-0.720269,0.793260,0.951145,-0.3,-0.995665,-0.434290,1.344650,0.429241
4,0,47,4,0.272025,1.074580,1.0,-0.136283,0.398579,0.044877,0.560109,-0.541985,-0.9,1.055636,0.812631,0.123457,-0.223359


In [5]:
labels.head()


,sequence,state
0,0,0
1,1,1
2,2,1
3,3,1
4,4,1


In [6]:
test.head()


,sequence,subject,step,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,sensor_09,sensor_10,sensor_11,sensor_12
0,25968,684,0,2.427357,19.639706,1.0,-1.466372,-1.289973,-4.207928,2.486339,-2.493893,8.0,-1.123555,-1.673048,10.980453,0.419011
1,25968,684,1,-4.950541,-21.747899,1.0,0.983186,-0.569053,1.845924,-3.887978,1.727481,-2.9,0.395231,-0.882233,-1.871399,-0.008525
2,25968,684,2,1.136012,-10.756303,1.0,1.016814,0.964157,2.454749,0.312386,1.154198,-5.6,1.114162,1.525273,-11.584362,0.139812
3,25968,684,3,0.806028,6.504202,1.0,-0.179646,0.969221,-1.035153,-0.457195,0.254962,-2.7,-0.588873,0.608761,-4.241770,-0.462916
4,25968,684,4,1.288253,5.552521,1.0,-0.493805,-1.036124,-1.126402,2.008197,-0.730534,0.0,0.899566,-1.259615,-0.472222,-0.121483


In [7]:
train.describe().round(2)


,sequence,subject,step,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,sensor_09,sensor_10,sensor_11,sensor_12
count,1558080.00,1558080.00,1558080.00,1558080.00,1558080.00,1558080.00,1558080.00,1558080.00,1558080.00,1558080.00,1558080.00,1558080.0,1558080.00,1558080.00,1558080.00,1558080.00
mean,12983.50,331.63,29.50,0.00,-0.00,-0.22,-0.00,-0.00,-0.00,-0.00,-0.00,-0.0,0.00,0.00,-0.00,-0.01
std,7496.32,195.83,17.32,2.66,4.40,2.30,3.93,1.68,1.59,3.35,3.24,4.5,2.59,1.92,4.53,39.12
min,0.00,0.00,0.00,-375.06,-434.60,-31.66,-408.38,-23.63,-74.98,-470.50,-407.01,-536.1,-270.35,-43.41,-427.06,-612.55
25%,6491.75,161.75,14.75,-0.50,-0.48,-0.65,-0.49,-0.47,-0.48,-0.49,-0.50,-0.5,-0.52,-0.48,-0.48,-0.58
50%,12983.50,335.00,29.50,-0.00,0.00,0.00,0.00,-0.00,0.00,0.00,-0.00,0.0,-0.00,-0.00,0.00,0.00
75%,19475.25,501.00,44.25,0.48,0.49,0.33,0.49,0.47,0.51,0.49,0.48,0.5,0.51,0.48,0.49,0.57
max,25967.00,671.00,59.00,335.82,449.59,1.67,436.65,24.87,77.92,442.50,331.25,630.1,367.98,41.87,448.02,630.51


### Data prep

In [8]:
train = train.reset_index(drop=True).merge(labels, on="sequence")


In [9]:
train["sequence_in_file"] = train["sequence"] - train["sequence"].min()
test["sequence_in_file"] = test["sequence"] - test["sequence"].min()

ctd = ["sequence", "subject", "step"]
train.drop(ctd, axis=1, inplace=True)
test.drop(ctd, axis=1, inplace=True)


In [10]:
test.columns, train.columns


(Index(['sensor_00', 'sensor_01', 'sensor_02', 'sensor_03', 'sensor_04',
        'sensor_05', 'sensor_06', 'sensor_07', 'sensor_08', 'sensor_09',
        'sensor_10', 'sensor_11', 'sensor_12', 'sequence_in_file'],
       dtype='object'),
 Index(['sensor_00', 'sensor_01', 'sensor_02', 'sensor_03', 'sensor_04',
        'sensor_05', 'sensor_06', 'sensor_07', 'sensor_08', 'sensor_09',
        'sensor_10', 'sensor_11', 'sensor_12', 'state', 'sequence_in_file'],
       dtype='object'))

In [11]:
col_names = [col for col in train if "sensor" in col]

In [12]:
s = StandardScaler()
X_train = train.drop(["state"], axis=1)
X_train[col_names] = s.fit_transform(X_train[col_names])
y_train = train[["state"]]
X_test = test.copy()
X_test[col_names] = s.transform(X_test[col_names])

In [30]:
class DatasetGenerator(Dataset):
    """inherit from the dataset class, pytorch needs len and get item wh"""

    def __init__(self, time_series, labels=None):
        super().__init__()
        self.time_series = time_series
        self.labels = labels
        self.cols = [col for col in self.time_series if "sensor" in col]

    def __len__(self):
        if self.labels is None:
            return len(self.time_series) // 60
        return len(self.labels)

    def __getitem__(self, index):
        if self.labels is None:
            label_tensor = np.nan
        else:
            label = self.labels[self.labels.index == index].state.iloc[0]
            label_tensor = torch.tensor(label, dtype=torch.long)

        ts_df = self.time_series[self.time_series.sequence_in_file == index][
            self.cols
        ].to_numpy()
        sensor_array = torch.tensor(ts_df, dtype=torch.float32)

        return sensor_array, label_tensor


In [31]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [43]:
class LSTM(nn.Module):
    def __init__(
        self,
        seq_length=60,
        input_dim=X_train.shape[1]-1,
        lstm_dim=512,
        num_layers=2,  # i.e 2 lstm models
        dropout=0.1,
        classes=2,
    ):

        super().__init__()

        self.LSTM0 = nn.LSTM(
            input_dim,
            lstm_dim,
            num_layers,
            batch_first=True,
            dropout=dropout,
            bidirectional=True,
        )

        self.logits = nn.Sequential(
            nn.Linear(2 * lstm_dim * 60, 128),
            nn.ReLU(),
            nn.Linear(128, classes),
        )

    def forward(self, x):
        out = self.LSTM0(x)[0].flatten(start_dim = 1)
        out = self.logits(out)
        return out


In [44]:
X_train

,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,sensor_09,sensor_10,sensor_11,sensor_12,sequence_in_file
0,-0.073994,0.025755,0.529941,0.084226,-0.595617,-0.081710,-0.037993,0.113685,-0.022186,-0.372260,-0.513943,0.118173,0.121766,0
1,-0.168462,0.030764,0.529941,-0.166807,0.097598,0.214962,-0.062496,-0.267356,0.044458,-0.116728,0.042977,-0.050247,0.011916,0
2,0.122789,-0.157416,0.529941,0.084451,0.282421,0.805957,-0.028192,0.165228,0.311034,0.385976,0.234122,-0.128556,-0.120775,0
3,0.196619,0.170766,0.529941,0.248882,-0.333470,-0.451728,0.237261,0.293261,-0.066615,-0.384521,-0.226680,0.297488,0.011273,0
4,0.102151,0.244225,0.529941,-0.034093,0.237816,0.029248,0.167563,-0.167094,-0.199903,0.406597,0.423661,0.028061,-0.005410,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1558075,-0.106549,-0.276191,-0.630265,0.149683,-0.551688,-0.282470,-0.194542,0.113685,0.088887,0.002817,-0.377539,-0.077258,-0.023085,25967
1558076,0.048959,0.079657,-0.630265,-0.082005,0.461579,-0.208184,0.029800,0.037665,-0.044400,0.248039,0.360436,-0.134457,-0.012885,25967
1558077,-0.218166,0.097783,-0.630265,0.081752,0.184530,0.178760,-0.153159,0.003774,-0.355406,-0.164101,0.373708,0.360135,0.024033,25967
1558078,0.480893,0.388758,-0.567891,0.204569,-0.272446,-0.033754,0.719161,0.013423,0.422107,0.108987,-0.477354,0.081175,0.005705,25967


In [45]:
train_dls = DatasetGenerator(X_train, y_train)
test_dls = DatasetGenerator(X_test)
train_loader = DataLoader(train_dls, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dls, batch_size=1, shuffle=False)


In [46]:
model = LSTM()
model = model.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0005)
loss_func = nn.CrossEntropyLoss()

In [47]:
EPOCHS = 100
loss_store = []
for e in range(EPOCHS):
    for x, y in train_loader:
        optimizer.zero_grad()
        x=x.to(device)
        y=y.to(device)
        preds = model(x)
        loss = loss_func(preds, y)
        loss_store.append(loss.item())
        loss.backward() # backprop 
        optimizer.step()



RuntimeError: stack expects each tensor to be equal size, but got [0, 13] at entry 0 and [60, 13] at entry 51

In [ ]:
def train(epochs, model, optimizer, criterion, scheduler, trainiter, validiter):

    for epoch in range(1, epochs + 1):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
    pass


In [29]:
y_train[y_train.index == 3]

,state
3,0
